# 第三章：複合型態（Composite Types）- 專業完整版

本章深入探討 Go 語言的五種複合型態：**陣列 (Array)**、**切片 (Slice)**、**字串 (String)**、**映射 (Map)** 和 **結構 (Struct)**。

## 學習重點
- 理解每種複合型態的內部結構與記憶體模型
- 掌握各種宣告、初始化和使用方式
- 學會避免常見陷阱和效能問題
- 了解最佳實務和設計模式

---

## 1. 陣列 (Array) - 固定長度的集合

陣列是 Go 中最基礎的集合型態，具有以下特性：
- **固定長度**：宣告後長度無法改變
- **同質元素**：所有元素必須是相同型態
- **值語意**：賦值和傳遞時會完整複製
- **長度是型態的一部分**：`[3]int` 和 `[5]int` 是不同的型態
- **記憶體連續**：元素在記憶體中連續存放

### 陣列的各種宣告方法

Go 提供了多種陣列宣告方式，讓開發者能根據不同需求選擇合適的初始化方法：

**基本宣告**: 最簡單的零值初始化方式，所有元素自動設為該型態的零值。
```go
var arr [5]int  // [0 0 0 0 0]
```

**指定初值**: 在宣告時直接設定陣列內容。
```go
arr := [3]int{1, 2, 3}
```

**自動推導長度**: 使用 `...` 讓編譯器根據初值數量自動決定陣列長度。
```go
arr := [...]string{"Go", "Python", "Java"}  // 長度為 3
```

**指定索引初始化**: 可以只初始化特定位置的元素，其他位置使用零值。
```go
arr := [5]int{1: 100, 3: 300}  // [0 100 0 300 0]
```

In [1]:
//go:build ignore
package main

import "fmt"

func main() {
    // 1. 基本宣告 - 零值初始化
    var arr1 [5]int // 所有元素初始化為 0
    fmt.Printf("零值陣列: %v\n", arr1)
    
    // 2. 宣告時指定初值
    var arr2 [3]int = [3]int{1, 2, 3}
    fmt.Printf("指定初值: %v\n", arr2)
    
    // 3. 短變數宣告
    arr3 := [4]string{"Go", "Python", "Java", "Rust"}
    fmt.Printf("短變數宣告: %v\n", arr3)
    
    // 4. 自動推導長度 - 使用 ...
    arr4 := [...]int{10, 20, 30, 40, 50}
    fmt.Printf("自動推導長度: %v (長度: %d)\n", arr4, len(arr4))
    
    // 5. 指定索引初始化
    arr5 := [5]int{1: 100, 3: 300} // 索引 1 和 3 設值，其他為零值
    fmt.Printf("指定索引: %v\n", arr5)
    
    // 6. 混合初始化
    arr6 := [...]string{"first", 2: "third", "fourth"} // 順序 + 指定索引
    fmt.Printf("混合初始化: %v\n", arr6)
}

零值陣列: [0 0 0 0 0]
指定初值: [1 2 3]
短變數宣告: [Go Python Java Rust]
自動推導長度: [10 20 30 40 50] (長度: 5)
指定索引: [0 100 0 300 0]
混合初始化: [first  third fourth]


### 陣列的操作與特性

In [ ]:
//go:build ignore
package main

import "fmt"

// 值傳遞函數 - 接收陣列副本
func modifyArrayValue(arr [3]int) {
    arr[0] = 999 // 只修改副本，不影響原陣列
    fmt.Printf("函數內修改後: %v\n", arr)
}

// 指標傳遞函數 - 接收陣列指標
func modifyArrayPointer(arr *[3]int) {
    (*arr)[0] = 888 // 透過指標修改原陣列
    fmt.Printf("指標函數內: %v\n", *arr)
}

func main() {
    // 1. 陣列的值語意
    original := [3]int{1, 2, 3}
    copied := original // 完整複製
    copied[0] = 100
    
    fmt.Printf("原始陣列: %v\n", original)
    fmt.Printf("複製陣列: %v\n", copied)
    
    // 2. 函數參數傳遞
    fmt.Println("\n=== 值傳遞 ===")
    fmt.Printf("呼叫前: %v\n", original)
    modifyArrayValue(original)
    fmt.Printf("呼叫後: %v\n", original) // 不變
    
    fmt.Println("\n=== 指標傳遞 ===")
    fmt.Printf("呼叫前: %v\n", original)
    modifyArrayPointer(&original)
    fmt.Printf("呼叫後: %v\n", original) // 已改變
    
    // 3. 陣列比較
    arr1 := [3]int{1, 2, 3}
    arr2 := [3]int{1, 2, 3}
    arr3 := [3]int{1, 2, 4}
    
    fmt.Printf("\narr1 == arr2: %v\n", arr1 == arr2) // true
    fmt.Printf("arr1 == arr3: %v\n", arr1 == arr3)   // false
    
    // 4. 陣列遍歷
    fruits := [4]string{"apple", "banana", "cherry", "date"}
    
    // 傳統 for 迴圈
    fmt.Println("\n傳統 for 迴圈:")
    for i := 0; i < len(fruits); i++ {
        fmt.Printf("  [%d]: %s\n", i, fruits[i])
    }
    
    // range 迴圈
    fmt.Println("range 迴圈:")
    for index, value := range fruits {
        fmt.Printf("  [%d]: %s\n", index, value)
    }
}

### 多維陣列

多維陣列是陣列的陣列，常用於表示矩陣、表格或其他多維資料結構。

**宣告語法**: `[行數][列數]型態`，例如 `[3][4]int` 表示 3 行 4 列的整數矩陣。

**記憶體配置**: Go 的多維陣列在記憶體中是連續存放的，按照行主序（row-major order）排列。

**初始化方式**: 
- 零值初始化：`var matrix [2][3]int`
- 完整初始化：`matrix := [2][3]int{{1,2,3}, {4,5,6}}`
- 部分初始化：某些行可以省略，自動填入零值

**訪問元素**: 使用 `matrix[i][j]` 的語法訪問第 i 行第 j 列的元素。

**遍歷方式**: 可以使用巢狀的 `for` 迴圈或 `range` 來遍歷所有元素。

簡單範例：
```go
// 建立 2x3 矩陣
matrix := [2][3]int{
    {1, 2, 3},
    {4, 5, 6},
}
fmt.Println(matrix[0][1])  // 輸出: 2
```

In [ ]:
//go:build ignore
package main

import "fmt"

func main() {
    // 1. 二維陣列宣告
    var matrix1 [3][4]int // 3x4 的零值矩陣
    fmt.Printf("零值二維陣列:\n%v\n\n", matrix1)
    
    // 2. 初始化二維陣列
    matrix2 := [2][3]int{
        {1, 2, 3},
        {4, 5, 6},
    }
    fmt.Printf("初始化二維陣列:\n%v\n\n", matrix2)
    
    // 3. 部分初始化
    matrix3 := [3][2]int{
        {10, 20},
        {}, // 零值行
        {50, 60},
    }
    fmt.Printf("部分初始化:\n%v\n\n", matrix3)
    
    // 4. 遍歷二維陣列
    fmt.Println("遍歷二維陣列:")
    for i, row := range matrix2 {
        for j, val := range row {
            fmt.Printf("  [%d][%d] = %d\n", i, j, val)
        }
    }
    
    // 5. 三維陣列
    cube := [2][2][2]int{
        {
            {1, 2},
            {3, 4},
        },
        {
            {5, 6},
            {7, 8},
        },
    }
    fmt.Printf("\n三維陣列: %v\n", cube)
    fmt.Printf("cube[1][0][1] = %d\n", cube[1][0][1]) // 訪問特定元素
}

### 陣列使用場景與注意事項

**適用場景：**
- 固定大小的資料集合（如座標、顏色值）
- 需要在編譯時確定大小的場合
- 與 C 語言互操作
- 高效能要求且大小固定的場合

**注意事項：**
- 大型陣列的值傳遞會複製大量資料，考慮使用指標
- 不同長度的陣列是不同型態，無法互相賦值
- 實際開發中，slice 更常用且靈活

## 2. 切片 (Slice) - 動態陣列

Slice 是 Go 中最重要的集合型態，內部結構包含三個欄位：
- **ptr**: 指向底層陣列的指標
- **len**: 目前長度
- **cap**: 容量（從指標位置到底層陣列結尾的長度）

### Slice 的各種宣告方法

Slice 提供了比陣列更靈活的宣告和初始化方式，適應不同的使用場景：

**nil slice**: 零值宣告，slice 為 nil，長度和容量都是 0。
```go
var slice []int  // nil slice
```

**空 slice**: 長度為 0 但不是 nil 的 slice。
```go
slice := []int{}
```

**make 函數**: 建立指定長度和容量的 slice。
```go
slice := make([]int, 5)       // 長度=容量=5
slice := make([]int, 3, 10)   // 長度=3, 容量=10
```

**字面值初始化**: 直接指定 slice 的內容。
```go
slice := []string{"Go", "Python", "JavaScript"}
```

**從陣列或其他 slice 建立**: 透過切片操作建立新的 slice。
```go
arr := [5]int{1, 2, 3, 4, 5}
slice := arr[:3]  // 取前 3 個元素
```

In [ ]:
//go:build ignore
package main

import "fmt"

func main() {
    // 1. 零值宣告 - nil slice
    var slice1 []int
    fmt.Printf("nil slice: %v, len=%d, cap=%d, is nil=%v\n", 
               slice1, len(slice1), cap(slice1), slice1 == nil)
    
    // 2. 空 slice 字面值
    slice2 := []int{}
    fmt.Printf("空 slice: %v, len=%d, cap=%d, is nil=%v\n", 
               slice2, len(slice2), cap(slice2), slice2 == nil)
    
    // 3. 帶初值的 slice
    slice3 := []string{"Go", "Python", "JavaScript"}
    fmt.Printf("初值 slice: %v, len=%d, cap=%d\n", 
               slice3, len(slice3), cap(slice3))
    
    // 4. 使用 make 建立 slice
    // make([]T, len) - 長度和容量相等
    slice4 := make([]int, 5)
    fmt.Printf("make(len): %v, len=%d, cap=%d\n", 
               slice4, len(slice4), cap(slice4))
    
    // make([]T, len, cap) - 指定長度和容量
    slice5 := make([]int, 3, 10)
    fmt.Printf("make(len,cap): %v, len=%d, cap=%d\n", 
               slice5, len(slice5), cap(slice5))
    
    // 5. 從陣列建立 slice
    array := [5]int{1, 2, 3, 4, 5}
    slice6 := array[:3] // 取前 3 個元素
    fmt.Printf("從陣列: %v, len=%d, cap=%d\n", 
               slice6, len(slice6), cap(slice6))
    
    // 6. 從其他 slice 建立
    slice7 := slice3[1:] // 從索引 1 開始
    fmt.Printf("子 slice: %v, len=%d, cap=%d\n", 
               slice7, len(slice7), cap(slice7))
}

### Slice 的切割操作

Slice 的切割（slicing）是其最強大的功能之一，允許從現有的 slice 或陣列建立新的 slice 視圖。

**基本語法**: `s[low:high]` 建立從索引 low 到 high-1 的子 slice。

**省略語法**: 
- `s[:high]` 從開頭到 high-1
- `s[low:]` 從 low 到結尾  
- `s[:]` 完整複製（但共享底層陣列）

**完整語法**: `s[low:high:max]` 可以限制新 slice 的容量，其中 `cap = max - low`。

**共享底層陣列**: 這是 slice 切割最需要注意的特性，子 slice 與原 slice 共享底層陣列，修改一個會影響另一個。

**防止共享**: 使用完整語法限制容量，或使用 `copy` 函數建立獨立的 slice。

範例：
```go
numbers := []int{0, 1, 2, 3, 4, 5}
sub := numbers[1:4]     // [1, 2, 3]
safe := numbers[1:4:4]  // 限制容量，防止意外擴展
```

In [2]:
//go:build ignore
package main

import "fmt"

func main() {
    // 基礎 slice
    numbers := []int{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    fmt.Printf("原始 slice: %v\n", numbers)
    
    // 1. 基本切割 s[low:high]
    fmt.Println("\n=== 基本切割 ===")
    fmt.Printf("numbers[2:5]: %v\n", numbers[2:5])   // [2 3 4]
    fmt.Printf("numbers[:4]: %v\n", numbers[:4])     // [0 1 2 3]
    fmt.Printf("numbers[6:]: %v\n", numbers[6:])     // [6 7 8 9]
    fmt.Printf("numbers[:]: %v\n", numbers[:])       // 完整複製
    
    // 2. 完整切割語法 s[low:high:max]
    fmt.Println("\n=== 完整切割語法 ===")
    slice1 := numbers[2:5:7] // len=5-2=3, cap=7-2=5
    fmt.Printf("numbers[2:5:7]: %v, len=%d, cap=%d\n", 
               slice1, len(slice1), cap(slice1))
    
    // 3. 共享底層陣列的影響
    fmt.Println("\n=== 共享底層陣列 ===")
    original := []int{1, 2, 3, 4, 5}
    sub := original[1:3] // [2, 3]
    
    fmt.Printf("原始: %v\n", original)
    fmt.Printf("子slice: %v\n", sub)
    
    // 修改子 slice
    sub[0] = 99
    fmt.Printf("修改後原始: %v\n", original) // 也被修改
    fmt.Printf("修改後子slice: %v\n", sub)
    
    // 4. 防止共享的方法 - 使用完整切割語法
    fmt.Println("\n=== 防止共享 ===")
    original2 := []int{10, 20, 30, 40, 50}
    safe := original2[1:3:3] // 限制容量，防止 append 覆蓋
    
    fmt.Printf("安全切割: %v, cap=%d\n", safe, cap(safe))
    safe = append(safe, 99) // 會重新分配記憶體
    fmt.Printf("append後原始: %v\n", original2) // 不受影響
    fmt.Printf("append後安全: %v\n", safe)
}

原始 slice: [0 1 2 3 4 5 6 7 8 9]

=== 基本切割 ===
numbers[2:5]: [2 3 4]
numbers[:4]: [0 1 2 3]
numbers[6:]: [6 7 8 9]
numbers[:]: [0 1 2 3 4 5 6 7 8 9]

=== 完整切割語法 ===
numbers[2:5:7]: [2 3 4], len=3, cap=5

=== 共享底層陣列 ===
原始: [1 2 3 4 5]
子slice: [2 3]
修改後原始: [1 99 3 4 5]
修改後子slice: [99 3]

=== 防止共享 ===
安全切割: [20 30], cap=2
append後原始: [10 20 30 40 50]
append後安全: [20 30 99]


### Slice 的 append 和 copy 操作

`append` 和 `copy` 是操作 slice 的兩個核心內建函數，提供了動態增長和複製的功能。

**append 函數**: 在 slice 末尾添加一個或多個元素，是 slice 動態特性的關鍵。
- 語法：`append(slice, element...)` 或 `append(slice, otherSlice...)`
- 自動擴容：當容量不足時，Go 會自動分配更大的底層陣列
- 容量增長策略：通常按倍數增長（如 2 倍），以減少重新分配的次數

**copy 函數**: 在兩個 slice 之間複製元素，確保資料獨立性。
- 語法：`copy(dst, src)` 回傳複製的元素數量
- 複製數量：取 `len(dst)` 和 `len(src)` 的最小值
- 安全性：即使 src 和 dst 重疊也能正確工作

**常見陷阱**: append 可能會重新分配記憶體，導致原本的 slice 引用失效。

範例：
```go
slice := []int{1, 2, 3}
slice = append(slice, 4, 5)      // 添加多個元素
copied := make([]int, len(slice))
copy(copied, slice)              // 建立獨立副本
```

In [3]:
//go:build ignore
package main

import "fmt"

func main() {
    // 1. append 基本使用
    fmt.Println("=== append 基本使用 ===")
    var slice []int
    fmt.Printf("初始: %v, len=%d, cap=%d\n", slice, len(slice), cap(slice))
    
    // 單個元素 append
    slice = append(slice, 1)
    fmt.Printf("append 1: %v, len=%d, cap=%d\n", slice, len(slice), cap(slice))
    
    // 多個元素 append
    slice = append(slice, 2, 3, 4)
    fmt.Printf("append 2,3,4: %v, len=%d, cap=%d\n", slice, len(slice), cap(slice))
    
    // append 另一個 slice
    other := []int{5, 6, 7}
    slice = append(slice, other...) // 展開操作符
    fmt.Printf("append slice: %v, len=%d, cap=%d\n", slice, len(slice), cap(slice))
    
    // 2. 容量增長觀察
    fmt.Println("\n=== 容量增長觀察 ===")
    s := make([]int, 0, 1)
    for i := 0; i < 10; i++ {
        oldCap := cap(s)
        s = append(s, i)
        if cap(s) != oldCap {
            fmt.Printf("容量從 %d 增長到 %d (添加元素 %d)\n", oldCap, cap(s), i)
        }
    }
    
    // 3. copy 函數使用
    fmt.Println("\n=== copy 函數使用 ===")
    src := []int{1, 2, 3, 4, 5}
    dst1 := make([]int, 3)
    dst2 := make([]int, 7)
    
    n1 := copy(dst1, src)
    n2 := copy(dst2, src)
    
    fmt.Printf("src: %v\n", src)
    fmt.Printf("dst1: %v, 複製了 %d 個元素\n", dst1, n1)
    fmt.Printf("dst2: %v, 複製了 %d 個元素\n", dst2, n2)
    
    // 4. 完全獨立的 slice 複製
    fmt.Println("\n=== 獨立複製 ===")
    original := []int{10, 20, 30}
    independent := make([]int, len(original))
    copy(independent, original)
    
    // 修改不會互相影響
    independent[0] = 999
    fmt.Printf("原始: %v\n", original)
    fmt.Printf("獨立: %v\n", independent)
    
    // 5. slice 的常見陷阱
    fmt.Println("\n=== 常見陷阱 ===")
    base := []int{1, 2, 3, 4, 5, 6}
    sub1 := base[1:3] // [2, 3]
    sub2 := base[2:4] // [3, 4]
    
    fmt.Printf("base: %v\n", base)
    fmt.Printf("sub1: %v\n", sub1)
    fmt.Printf("sub2: %v\n", sub2)
    
    // 透過 sub1 修改會影響 base 和 sub2
    sub1[1] = 99 // 修改索引 2 的元素
    fmt.Printf("修改 sub1[1] 後:\n")
    fmt.Printf("base: %v\n", base)
    fmt.Printf("sub1: %v\n", sub1)
    fmt.Printf("sub2: %v\n", sub2) // 第一個元素被改變
}

=== append 基本使用 ===
初始: [], len=0, cap=0
append 1: [1], len=1, cap=1
append 2,3,4: [1 2 3 4], len=4, cap=4
append slice: [1 2 3 4 5 6 7], len=7, cap=8

=== 容量增長觀察 ===
容量從 1 增長到 2 (添加元素 1)
容量從 2 增長到 4 (添加元素 2)
容量從 4 增長到 8 (添加元素 4)
容量從 8 增長到 16 (添加元素 8)

=== copy 函數使用 ===
src: [1 2 3 4 5]
dst1: [1 2 3], 複製了 3 個元素
dst2: [1 2 3 4 5 0 0], 複製了 5 個元素

=== 獨立複製 ===
原始: [10 20 30]
獨立: [999 20 30]

=== 常見陷阱 ===
base: [1 2 3 4 5 6]
sub1: [2 3]
sub2: [3 4]
修改 sub1[1] 後:
base: [1 2 99 4 5 6]
sub1: [2 99]
sub2: [99 4]


### Slice 最佳實務與效能考量

掌握 slice 的效能特性和最佳實務，能夠寫出更高效的 Go 程式。

**預分配容量**: 當知道大概需要多少元素時，使用 `make([]T, 0, capacity)` 預分配容量，避免多次記憶體重分配。

**元素移除策略**: 
- 保持順序：`append(slice[:i], slice[i+1:]...)`（O(n) 時間複雜度）
- 不保持順序：將最後元素移到要刪除的位置（O(1) 時間複雜度）

**過濾操作**: 使用原地過濾技巧 `slice[:0]` 重用底層陣列，減少記憶體分配。

**避免記憶體洩漏**: 當從大 slice 中取小部分時，考慮使用 `copy` 建立獨立的 slice，讓大 slice 能被垃圾回收。

**併發安全**: slice 的 append 操作不是併發安全的，多個 goroutine 同時操作需要適當同步。

效能對比範例：
```go
// 低效：未預分配容量，多次重分配
var s1 []int
for i := 0; i < 10000; i++ {
    s1 = append(s1, i)
}

// 高效：預分配容量，一次分配
s2 := make([]int, 0, 10000)
for i := 0; i < 10000; i++ {
    s2 = append(s2, i)
}
```

In [ ]:
//go:build ignore
package main

import (
    "fmt"
    "time"
)

func main() {
    // 1. 預先分配容量的效能優勢
    fmt.Println("=== 效能比較 ===")
    
    // 不預分配容量
    start := time.Now()
    var s1 []int
    for i := 0; i < 100000; i++ {
        s1 = append(s1, i)
    }
    duration1 := time.Since(start)
    
    // 預分配容量
    start = time.Now()
    s2 := make([]int, 0, 100000)
    for i := 0; i < 100000; i++ {
        s2 = append(s2, i)
    }
    duration2 := time.Since(start)
    
    fmt.Printf("不預分配: %v\n", duration1)
    fmt.Printf("預分配: %v\n", duration2)
    fmt.Printf("效能提升: %.2fx\n", float64(duration1)/float64(duration2))
    
    // 2. 移除元素的不同方法
    fmt.Println("\n=== 移除元素方法 ===")
    
    // 移除第 i 個元素（保持順序）
    removeAtIndex := func(slice []int, index int) []int {
        return append(slice[:index], slice[index+1:]...)
    }
    
    // 移除第 i 個元素（不保持順序，更快） 把「最後一個元素」覆蓋到 index 位置
    removeAtIndexFast := func(slice []int, index int) []int {
        slice[index] = slice[len(slice)-1]
        return slice[:len(slice)-1]
    }
    
    data1 := []int{1, 2, 3, 4, 5}
    data2 := []int{1, 2, 3, 4, 5}
    
    fmt.Printf("原始: %v\n", data1)
    result1 := removeAtIndex(data1, 2)
    fmt.Printf("移除索引2(保序): %v\n", result1)
    
    fmt.Printf("原始: %v\n", data2)
    result2 := removeAtIndexFast(data2, 2)
    fmt.Printf("移除索引2(快速): %v\n", result2)
    
    // removeAtIndexFast 快速的原因：
    // 1. removeAtIndex: 需要將索引後的所有元素向前移動，時間複雜度 O(n)
    // 2. removeAtIndexFast: 只需要一次賦值操作，時間複雜度 O(1)
    //    - 將最後一個元素複製到要刪除的位置
    //    - 縮短 slice 長度
    //    - 不需要移動其他元素，但會改變元素順序
    fmt.Println("快速移除的代價是不保持原有順序")
    
    // 3. 過濾元素
    fmt.Println("\n=== 過濾元素 ===")
    numbers := []int{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    
    // 過濾出偶數（原地操作）
    evens := numbers[:0] // 重用底層陣列
    for _, n := range numbers {
        if n%2 == 0 {
            evens = append(evens, n)
        }
    }
    
    fmt.Printf("原始: %v\n", numbers)
    fmt.Printf("偶數: %v\n", evens)
    
    // 4. 反轉 slice
    fmt.Println("\n=== 反轉 slice ===")
    reverse := func(s []int) {
        for i, j := 0, len(s)-1; i < j; i, j = i+1, j-1 {
            s[i], s[j] = s[j], s[i]
        }
    }
    
    arr := []int{1, 2, 3, 4, 5}
    fmt.Printf("反轉前: %v\n", arr)
    reverse(arr)
    fmt.Printf("反轉後: %v\n", arr)
}

=== 效能比較 ===
不預分配: 1.25125ms
預分配: 179.083µs
效能提升: 6.99x

=== 移除元素方法 ===
原始: [1 2 3 4 5]
移除索引2(保序): [1 2 4 5]
原始: [1 2 3 4 5]
移除索引2(快速): [1 2 5 4]

=== 過濾元素 ===
原始: [2 4 6 8 10 6 7 8 9 10]
偶數: [2 4 6 8 10]

=== 反轉 slice ===
反轉前: [1 2 3 4 5]
反轉後: [5 4 3 2 1]


## 3. 字串、Rune 與 Byte

Go 的字串處理涉及三個重要概念：
- **string**: 不可變的 UTF-8 編碼位元組序列
- **byte**: uint8 的別名，代表一個位元組
- **rune**: int32 的別名，代表一個 Unicode 碼點

### 字串的基本特性

Go 的字串有幾個重要特性需要理解：

**不可變性**: 字串一旦建立就不能修改，任何「修改」操作都會建立新字串。

**UTF-8 編碼**: Go 的字串內部使用 UTF-8 編碼，這意味著一個字元可能佔用 1-4 個位元組。

**位元組 vs 字元**: `len()` 回傳位元組數，而 `utf8.RuneCountInString()` 回傳字元數。

**多種宣告方式**: 支援雙引號、反引號（原始字串）等不同的宣告方式。

簡單範例：
```go
text := "Hello, 世界!"  // 包含中文字元
fmt.Println(len(text))  // 位元組數：13
fmt.Println(utf8.RuneCountInString(text))  // 字元數：9
```

In [ ]:
//go:build ignore
package main

import (
    "fmt"
    "unicode/utf8"
)

func main() {
    // 1. 字串的不同宣告方式
    var str1 string = "Hello, World!"
    str2 := "Hello, 世界!"
    str3 := `多行字串
可以包含換行
和"引號"`
    
    fmt.Printf("str1: %s\n", str1)
    fmt.Printf("str2: %s\n", str2)
    fmt.Printf("str3: %s\n\n", str3)
    
    // 2. 字串長度的差異
    text := "Hello, 世界🌍!"
    fmt.Printf("字串: %s\n", text)
    fmt.Printf("len(位元組): %d\n", len(text))
    fmt.Printf("utf8.RuneCountInString(字元): %d\n\n", utf8.RuneCountInString(text))
    
    // 3. 字串的位元組和字元訪問
    fmt.Println("=== 位元組層級訪問 ===")
    for i := 0; i < len(text); i++ {
        fmt.Printf("byte[%d]: %d (%c)\n", i, text[i], text[i])
    }
    
    fmt.Println("\n=== 字元層級訪問 ===")
    for i, r := range text {
        fmt.Printf("rune[%d]: %d (%c) U+%04X\n", i, r, r, r)
    }
    
    // 4. 字串與 []byte 和 []rune 的轉換
    fmt.Println("\n=== 型別轉換 ===")
    
    // string -> []byte
    bytes := []byte(text)
    fmt.Printf("[]byte: %v\n", bytes)
    
    // string -> []rune
    runes := []rune(text)
    fmt.Printf("[]rune: %v\n", runes)
    
    // []byte -> string
    str4 := string(bytes)
    fmt.Printf("from bytes: %s\n", str4)
    
    // []rune -> string
    str5 := string(runes)
    fmt.Printf("from runes: %s\n", str5)
}

### 字串操作與處理

Go 提供了豐富的字串操作功能，主要透過 `strings` 套件實現：

**字串不可變的影響**: 由於字串不可變，修改字串需要建立新的字串或轉換為可變的 slice。

**高效拼接**: 對於大量字串拼接，應使用 `strings.Builder` 而非 `+` 操作符。

**常用操作**: 包括去空白、大小寫轉換、包含檢查、前後綴判斷等。

**分割與合併**: `strings.Split()` 和 `strings.Join()` 是處理字串陣列的常用方法。

**搜尋與替換**: 支援查找字串位置、計算出現次數、替換指定內容等。

範例：
```go
// 高效拼接
var builder strings.Builder
builder.WriteString("Hello")
builder.WriteString(" World")
result := builder.String()
```

In [ ]:
//go:build ignore
package main

import (
    "fmt"
    "strings"
    "unicode/utf8"
)

func main() {
    // 1. 字串不可變性
    fmt.Println("=== 字串不可變性 ===")
    original := "Hello"
    // original[0] = 'h' // 編譯錯誤！字串不可變
    
    // 需要透過轉換來修改
    runes := []rune(original)
    runes[0] = 'h'
    modified := string(runes)
    
    fmt.Printf("原始: %s\n", original)
    fmt.Printf("修改: %s\n\n", modified)
    
    // 2. 高效字串操作 - strings.Builder
    fmt.Println("=== 高效字串拼接 ===")
    
    // 低效方式（每次都創建新字串）
    var result1 string
    for i := 0; i < 5; i++ {
        result1 += fmt.Sprintf("item%d ", i)
    }
    fmt.Printf("低效拼接: %s\n", result1)
    
    // 高效方式
    var builder strings.Builder
    builder.Grow(50) // 預分配容量
    for i := 0; i < 5; i++ {
        fmt.Fprintf(&builder, "item%d ", i)
    }
    result2 := builder.String()
    fmt.Printf("高效拼接: %s\n\n", result2)
    
    // 3. 常用字串操作
    fmt.Println("=== 常用字串操作 ===")
    text := "  Hello, 世界! Welcome to Go!  "
    
    fmt.Printf("原始: '%s'\n", text)
    fmt.Printf("去空白: '%s'\n", strings.TrimSpace(text))
    fmt.Printf("轉大寫: '%s'\n", strings.ToUpper(text))
    fmt.Printf("轉小寫: '%s'\n", strings.ToLower(text))
    fmt.Printf("包含'Go': %v\n", strings.Contains(text, "Go"))
    fmt.Printf("前綴'  Hello': %v\n", strings.HasPrefix(text, "  Hello"))
    fmt.Printf("後綴'Go!  ': %v\n", strings.HasSuffix(text, "Go!  "))
    
    // 4. 字串分割與合併
    fmt.Println("\n=== 分割與合併 ===")
    sentence := "Go,Python,JavaScript,Rust"
    languages := strings.Split(sentence, ",")
    fmt.Printf("分割: %v\n", languages)
    
    joined := strings.Join(languages, " | ")
    fmt.Printf("合併: %s\n", joined)
    
    // 5. 字串搜尋與替換
    fmt.Println("\n=== 搜尋與替換 ===")
    content := "Go is great. Go is simple. Go is efficient."
    
    index := strings.Index(content, "Go")
    lastIndex := strings.LastIndex(content, "Go")
    count := strings.Count(content, "Go")
    
    fmt.Printf("內容: %s\n", content)
    fmt.Printf("首次出現位置: %d\n", index)
    fmt.Printf("最後出現位置: %d\n", lastIndex)
    fmt.Printf("出現次數: %d\n", count)
    
    replaced := strings.Replace(content, "Go", "Golang", 2) // 替換前2個
    replaceAll := strings.ReplaceAll(content, "Go", "Golang") // 全部替換
    
    fmt.Printf("替換前2個: %s\n", replaced)
    fmt.Printf("全部替換: %s\n", replaceAll)
}

### Unicode 和 UTF-8 處理

Go 的字串處理深度整合了 Unicode 標準和 UTF-8 編碼，提供了強大的國際化支援。

**UTF-8 編碼特性**: UTF-8 是可變長度編碼，一個 Unicode 字元可能占用 1-4 個位元組：
- ASCII 字元（U+0000 到 U+007F）：1 位元組
- 拉丁字母擴展、希臘文等（U+0080 到 U+07FF）：2 位元組  
- 中日韓文字（U+0800 到 U+FFFF）：3 位元組
- 表情符號等（U+10000 以上）：4 位元組

**長度差異**: `len(string)` 回傳位元組數，`utf8.RuneCountInString()` 回傳字元數。

**字元分類**: `unicode` 套件提供豐富的字元分類函數，如 `IsLetter`、`IsDigit`、`IsSpace` 等。

**安全的字串操作**: 直接用位元組索引切割字串可能會破壞 UTF-8 序列，應該使用 rune 切片進行安全操作。

**編碼驗證**: `utf8.ValidString()` 可以檢查字串是否為有效的 UTF-8 序列。

範例：
```go
text := "Hello, 世界🌍!"
fmt.Println(len(text))                    // 位元組數: 15  
fmt.Println(utf8.RuneCountInString(text)) // 字元數: 10

// 安全的字元切片
runes := []rune(text)
substr := string(runes[7:9])  // "世界"
```

In [ ]:
//go:build ignore
package main

import (
    "fmt"
    "unicode"
    "unicode/utf8"
)

func main() {
    // 1. UTF-8 編碼分析
    fmt.Println("=== UTF-8 編碼分析 ===")
    text := "A世🌍"
    
    fmt.Printf("字串: %s\n", text)
    fmt.Printf("位元組數: %d\n", len(text))
    fmt.Printf("字元數: %d\n\n", utf8.RuneCountInString(text))
    
    // 逐字元分析編碼
    for i, r := range text {
        size := utf8.RuneLen(r)
        fmt.Printf("字元 '%c' (U+%04X): 位置=%d, UTF-8長度=%d位元組\n", r, r, i, size)
    }
    
    // 2. UTF-8 驗證與修復
    fmt.Println("\n=== UTF-8 驗證 ===")
    
    validUTF8 := "Hello, 世界!"
    invalidUTF8 := string([]byte{0xff, 0xfe, 0xfd}) // 無效的UTF-8序列
    
    fmt.Printf("'%s' 是否有效UTF-8: %v\n", validUTF8, utf8.ValidString(validUTF8))
    fmt.Printf("無效序列是否有效UTF-8: %v\n", utf8.ValidString(invalidUTF8))
    
    // 3. 手動 UTF-8 解碼
    fmt.Println("\n=== 手動 UTF-8 解碼 ===")
    s := "Hello, 世界!"
    
    for len(s) > 0 {
        r, size := utf8.DecodeRuneInString(s)
        fmt.Printf("字元: '%c' (U+%04X), 大小: %d 位元組\n", r, r, size)
        s = s[size:]
    }
    
    // 4. Unicode 字元分類
    fmt.Println("\n=== Unicode 字元分類 ===")
    chars := []rune{'A', 'a', '中', '1', ' ', '!', '🌍'}
    
    for _, r := range chars {
        fmt.Printf("'%c' - ", r)
        
        if unicode.IsLetter(r) {
            fmt.Print("字母 ")
        }
        if unicode.IsDigit(r) {
            fmt.Print("數字 ")
        }
        if unicode.IsSpace(r) {
            fmt.Print("空白 ")
        }
        if unicode.IsPunct(r) {
            fmt.Print("標點 ")
        }
        if unicode.IsSymbol(r) {
            fmt.Print("符號 ")
        }
        if unicode.IsUpper(r) {
            fmt.Print("大寫 ")
        }
        if unicode.IsLower(r) {
            fmt.Print("小寫 ")
        }
        
        fmt.Println()
    }
    
    // 5. 安全的字串切片（按字元）
    fmt.Println("\n=== 安全的字串切片 ===")
    
    substringByRunes := func(s string, start, length int) string {
        runes := []rune(s)
        if start >= len(runes) {
            return ""
        }
        end := start + length
        if end > len(runes) {
            end = len(runes)
        }
        return string(runes[start:end])
    }
    
    original := "Hello, 世界! 🌍"
    sub1 := substringByRunes(original, 7, 2) // 取「世界」
    sub2 := substringByRunes(original, 0, 5) // 取前5個字元
    
    fmt.Printf("原始: %s\n", original)
    fmt.Printf("字元7-8: %s\n", sub1)
    fmt.Printf("前5字元: %s\n", sub2)
}

## 4. 映射 (Map) - 鍵值對集合

Map 是 Go 中的雜湊表實作，提供 O(1) 平均時間複雜度的鍵值對存取。

### Map 的各種宣告方法

Map 提供了靈活的宣告和初始化方式，適合不同的使用場景：

**零值宣告**: 宣告一個 nil map，只能讀取（回傳零值）但不能寫入。
```go
var m map[string]int  // nil map
fmt.Println(m["key"]) // 安全，回傳 0
// m["key"] = 1       // panic！
```

**make 函數**: 建立一個可用的空 map。
```go
m := make(map[string]int)
m["key"] = 42  // 正常工作
```

**字面值初始化**: 直接指定初始的鍵值對。
```go
colors := map[string]int{
    "red":   1,
    "green": 2,
    "blue":  3,
}
```

**複雜型態**: map 的鍵必須是可比較的型態，值可以是任意型態。
```go
// 結構當鍵
type Point struct{ X, Y int }
points := map[Point]string{
    {0, 0}: "origin",
    {1, 1}: "diagonal",
}
```

**注意事項**: 鍵的型態限制（不能是 slice、map、function），值沒有限制。

In [ ]:
//go:build ignore
package main

import "fmt"

func main() {
    // 1. 零值宣告 - nil map（只能讀不能寫）
    var m1 map[string]int
    fmt.Printf("nil map: %v, len=%d, is nil=%v\n", m1, len(m1), m1 == nil)
    
    // 讀取 nil map 是安全的，返回零值
    value := m1["key"]
    fmt.Printf("nil map 讀取: %d\n", value)
    
    // m1["key"] = 1 // 運行時 panic！不能寫入 nil map
    
    // 2. 使用 make 建立空 map
    m2 := make(map[string]int)
    m2["apple"] = 5
    m2["banana"] = 3
    fmt.Printf("make map: %v\n", m2)
    
    // 3. 帶初值的 map 字面值
    m3 := map[string]int{
        "red":    1,
        "green":  2,
        "blue":   3,
    }
    fmt.Printf("字面值 map: %v\n", m3)
    
    // 4. 空 map 字面值
    m4 := map[string]int{} // 等同於 make(map[string]int)
    fmt.Printf("空字面值 map: %v, is nil=%v\n", m4, m4 == nil)
    
    // 5. 指定容量的 map（提示，非強制）
    m5 := make(map[string]int, 100) // 預期有100個元素
    fmt.Printf("指定容量 map: %v\n", m5)
    
    // 6. 複雜型別的 map
    type Person struct {
        Name string
        Age  int
    }
    
    // map[鍵型別]值型別
    people := map[int]Person{
        1001: {Name: "Alice", Age: 25},
        1002: {Name: "Bob", Age: 30},
    }
    fmt.Printf("結構 map: %v\n", people)
    
    // 巢狀 map
    matrix := map[string]map[string]int{
        "row1": {"col1": 1, "col2": 2},
        "row2": {"col1": 3, "col2": 4},
    }
    fmt.Printf("巢狀 map: %v\n", matrix)
    
    // 7. 不同鍵型別的範例
    intKeyMap := map[int]string{1: "one", 2: "two"}
    floatKeyMap := map[float64]string{3.14: "pi", 2.71: "e"}
    
    // 結構當作鍵（必須可比較）
    type Point struct{ X, Y int }
    pointMap := map[Point]string{
        {0, 0}: "origin",
        {1, 1}: "diagonal",
    }
    
    fmt.Printf("int鍵: %v\n", intKeyMap)
    fmt.Printf("float鍵: %v\n", floatKeyMap)
    fmt.Printf("結構鍵: %v\n", pointMap)
}

### Map 的基本操作

Map 的操作包含讀取、寫入、刪除和遍歷，每種操作都有其特定的語法和注意事項。

**讀寫操作**: 使用方括號語法進行讀寫，讀取不存在的鍵會回傳零值。
```go
m := make(map[string]int)
m["apple"] = 5           // 寫入
value := m["apple"]      // 讀取存在的鍵
missing := m["orange"]   // 讀取不存在的鍵，回傳 0
```

**comma ok 模式**: 檢查鍵是否存在，避免零值的歧義。
```go
if value, exists := m["apple"]; exists {
    fmt.Printf("apple 存在，值為 %d\n", value)
}
```

**刪除操作**: 使用內建的 `delete` 函數，刪除不存在的鍵是安全的。
```go
delete(m, "apple")     // 刪除存在的鍵
delete(m, "nonexist")  // 刪除不存在的鍵，不會 panic
```

**遍歷**: 使用 `range` 遍歷，但順序不保證。
```go
for key, value := range m {
    fmt.Printf("%s: %d\n", key, value)
}
```

**長度與清空**: `len()` 取得元素數量，清空需要重新建立或逐一刪除。

In [ ]:
//go:build ignore
package main

import "fmt"

func main() {
    // 1. 基本讀寫操作
    fmt.Println("=== 基本讀寫操作 ===")
    scores := make(map[string]int)
    
    // 寫入
    scores["Alice"] = 95
    scores["Bob"] = 87
    scores["Charlie"] = 92
    
    // 讀取
    fmt.Printf("Alice的分數: %d\n", scores["Alice"])
    fmt.Printf("不存在的David: %d\n", scores["David"]) // 返回零值
    
    // 2. comma ok 模式檢查存在性
    fmt.Println("\n=== comma ok 模式 ===")
    
    if score, exists := scores["Alice"]; exists {
        fmt.Printf("Alice存在，分數: %d\n", score)
    }
    
    if score, exists := scores["David"]; !exists {
        fmt.Printf("David不存在\n")
    } else {
        fmt.Printf("David分數: %d\n", score)
    }
    
    // 3. 刪除操作
    fmt.Println("\n=== 刪除操作 ===")
    fmt.Printf("刪除前: %v\n", scores)
    
    delete(scores, "Bob")
    delete(scores, "NotExist") // 刪除不存在的鍵是安全的
    
    fmt.Printf("刪除後: %v\n", scores)
    
    // 4. 遍歷 map
    fmt.Println("\n=== 遍歷 map ===")
    
    // range 遍歷（順序不保證）
    for name, score := range scores {
        fmt.Printf("  %s: %d\n", name, score)
    }
    
    // 只遍歷鍵
    fmt.Println("只遍歷鍵:")
    for name := range scores {
        fmt.Printf("  %s\n", name)
    }
    
    // 只遍歷值
    fmt.Println("只遍歷值:")
    for _, score := range scores {
        fmt.Printf("  %d\n", score)
    }
    
    // 5. Map 的長度
    fmt.Printf("\nMap 長度: %d\n", len(scores))
    
    // 6. 清空 map（沒有內建 clear，需要重新建立或逐一刪除）
    fmt.Println("\n=== 清空 map ===")
    
    // 方法1: 重新建立
    scores = make(map[string]int)
    fmt.Printf("重新建立後: %v\n", scores)
    
    // 方法2: 逐一刪除（適用於想重用已分配的記憶體）
    scores["test1"] = 1
    scores["test2"] = 2
    
    for k := range scores {
        delete(scores, k)
    }
    fmt.Printf("逐一刪除後: %v\n", scores)
}

### Map 的進階使用與最佳實務

Map 在實際開發中有許多進階用法和設計模式，掌握這些技巧能提升程式碼品質。

**Set 實作**: 使用 `map[T]struct{}` 實作集合，`struct{}` 不占用記憶體。
```go
set := make(map[string]struct{})
set["item"] = struct{}{}  // 添加元素
if _, exists := set["item"]; exists {
    // 元素存在
}
```

**計數器模式**: Map 天然適合做計數統計。
```go
counter := make(map[rune]int)
for _, char := range "hello world" {
    counter[char]++
}
```

**分組操作**: 將資料按某個欄位分組。
```go
groups := make(map[string][]Student)
for _, student := range students {
    grade := student.Grade
    groups[grade] = append(groups[grade], student)
}
```

**有序遍歷**: Map 的遍歷順序不固定，需要排序時先收集鍵。
```go
keys := make([]string, 0, len(m))
for k := range m {
    keys = append(keys, k)
}
sort.Strings(keys)
for _, k := range keys {
    fmt.Printf("%s: %d\n", k, m[k])
}
```

**值為指標**: 當值是大型結構或需要修改時，考慮使用指標作為值。
```go
users := make(map[int]*User)
users[1] = &User{Name: "Alice"}
users[1].Age = 25  // 直接修改
```

In [ ]:
//go:build ignore
package main

import (
    "fmt"
    "sort"
)

func main() {
    // 1. 使用 map 實作 Set
    fmt.Println("=== Map 實作 Set ===")
    
    // 使用 struct{} 節省記憶體
    set := make(map[string]struct{})
    items := []string{"apple", "banana", "apple", "orange", "banana"}
    
    // 加入元素到集合
    for _, item := range items {
        set[item] = struct{}{}
    }
    
    fmt.Printf("原始切片: %v\n", items)
    fmt.Print("去重後集合: ")
    for item := range set {
        fmt.Printf("%s ", item)
    }
    fmt.Println()
    
    // 檢查元素是否在集合中
    if _, exists := set["apple"]; exists {
        fmt.Println("apple 在集合中")
    }
    
    // 2. 計數器模式
    fmt.Println("\n=== 計數器模式 ===")
    
    text := "hello world"
    charCount := make(map[rune]int)
    
    for _, char := range text {
        charCount[char]++
    }
    
    fmt.Printf("字元計數: %v\n", charCount)
    
    // 3. 分組操作
    fmt.Println("\n=== 分組操作 ===")
    
    type Student struct {
        Name  string
        Grade string
        Score int
    }
    
    students := []Student{
        {"Alice", "A", 95},
        {"Bob", "B", 87},
        {"Charlie", "A", 92},
        {"David", "B", 78},
    }
    
    // 按年級分組
    gradeGroups := make(map[string][]Student)
    
    for _, student := range students {
        gradeGroups[student.Grade] = append(gradeGroups[student.Grade], student)
    }
    
    for grade, group := range gradeGroups {
        fmt.Printf("年級 %s: %v\n", grade, group)
    }
    
    // 4. 有序遍歷 map
    fmt.Println("\n=== 有序遍歷 map ===")
    
    ages := map[string]int{
        "Charlie": 25,
        "Alice":   30,
        "Bob":     20,
        "David":   35,
    }
    
    // 收集鍵並排序
    var names []string
    for name := range ages {
        names = append(names, name)
    }
    sort.Strings(names)
    
    fmt.Println("按姓名排序:")
    for _, name := range names {
        fmt.Printf("  %s: %d\n", name, ages[name])
    }
    
    // 5. Map 的值是指標
    fmt.Println("\n=== Map 值是指標 ===")
    
    type Counter struct {
        Value int
    }
    
    counters := make(map[string]*Counter)
    
    // 建立和初始化
    counters["a"] = &Counter{Value: 1}
    counters["b"] = &Counter{Value: 2}
    
    // 透過指標修改
    counters["a"].Value = 10
    
    fmt.Printf("修改後: %v\n", counters["a"].Value)
    
    // 6. Map 的複製（淺複製）
    fmt.Println("\n=== Map 複製 ===")
    
    original := map[string]int{"a": 1, "b": 2}
    
    // 複製 map
    copied := make(map[string]int)
    for k, v := range original {
        copied[k] = v
    }
    
    // 修改複製的不影響原始
    copied["a"] = 10
    
    fmt.Printf("原始: %v\n", original)
    fmt.Printf("複製: %v\n", copied)
}

### Map 的限制與注意事項

了解 Map 的限制和潛在陷阱，有助於避免常見錯誤和效能問題。

**鍵的限制**: 鍵必須是可比較的型態，不能使用 slice、map 或 function。
```go
// 有效的鍵型態
map[string]int{}    // 字串
map[int]string{}    // 整數
map[[3]int]string{} // 陣列
map[*User]int{}     // 指標

// 無效的鍵型態（編譯錯誤）
// map[[]int]string{}    // slice
// map[map[string]int]string{} // map  
// map[func()]string{}   // function
```

**併發安全**: Map 不是併發安全的，多個 goroutine 同時讀寫會導致競態條件。
```go
// 需要使用 sync.Mutex 或 sync.RWMutex 保護
// 或者使用 sync.Map（適用於特定場景）
```

**nil Map 陷阱**: nil map 可以讀取但不能寫入。
```go
var m map[string]int  // nil map
value := m["key"]     // 安全，回傳 0
// m["key"] = 1       // runtime panic
```

**迭代順序**: Map 的迭代順序是不確定的，且 Go 1.0 之後故意隨機化。
```go
// 每次執行可能有不同的順序
for k, v := range m {
    fmt.Printf("%s: %d\n", k, v)
}
```

**記憶體管理**: Map 的記憶體不會自動縮小，即使刪除大量元素。
```go
// 如果需要釋放記憶體，可能需要重新建立 map
largeMap := make(map[int]string)
// ... 添加大量元素後再刪除
// 記憶體仍然被占用，考慮重新建立
largeMap = make(map[int]string)
```

In [ ]:
//go:build ignore
package main

import "fmt"

func main() {
    // 1. 鍵的限制 - 必須是可比較的型別
    fmt.Println("=== 鍵的限制 ===")
    
    // 可比較的型別：基本型別、陣列、指標、channel、介面
    validKeys := map[interface{}]string{
        "string":    "字串鍵",
        42:          "整數鍵",
        3.14:        "浮點數鍵",
        true:        "布林鍵",
        [3]int{1,2,3}: "陣列鍵",
    }
    
    fmt.Printf("有效鍵: %v\n", validKeys)
    
    // 不可比較的型別會導致編譯錯誤：
    // map[[]int]string{}     // slice 不能當鍵
    // map[map[string]int]string{} // map 不能當鍵
    // map[func()]string{}    // function 不能當鍵
    
    // 2. Map 不是執行緒安全的
    fmt.Println("\n=== 併發安全問題 ===")
    
    // 在實際應用中，需要使用 sync.Mutex 或 sync.RWMutex
    // 或使用 sync.Map（適用於特定場景）
    fmt.Println("Map 不是併發安全的，多個 goroutine 同時讀寫需要加鎖")
    
    // 3. Map 的零值是 nil
    fmt.Println("\n=== 零值問題 ===")
    
    var nilMap map[string]int
    fmt.Printf("nil map: %v\n", nilMap)
    
    // 讀取是安全的
    value := nilMap["key"]
    fmt.Printf("從 nil map 讀取: %d\n", value)
    
    // 寫入會 panic
    defer func() {
        if r := recover(); r != nil {
            fmt.Printf("寫入 nil map 會 panic: %v\n", r)
        }
    }()
    
    // nilMap["key"] = 1 // 這會 panic
    
    // 4. Map 的迭代順序不確定
    fmt.Println("\n=== 迭代順序不確定 ===")
    
    m := map[string]int{"a": 1, "b": 2, "c": 3}
    
    fmt.Println("多次迭代可能有不同順序:")
    for i := 0; i < 3; i++ {
        fmt.Printf("第%d次: ", i+1)
        for k, v := range m {
            fmt.Printf("%s:%d ", k, v)
        }
        fmt.Println()
    }
    
    // 5. Map 的記憶體不會自動縮小
    fmt.Println("\n=== 記憶體管理 ===")
    
    largeMap := make(map[int]string)
    
    // 添加大量元素
    for i := 0; i < 1000; i++ {
        largeMap[i] = fmt.Sprintf("value%d", i)
    }
    
    fmt.Printf("大量添加後長度: %d\n", len(largeMap))
    
    // 刪除大部分元素
    for i := 0; i < 950; i++ {
        delete(largeMap, i)
    }
    
    fmt.Printf("大量刪除後長度: %d\n", len(largeMap))
    fmt.Println("注意：Map 的底層記憶體不會自動釋放")
    fmt.Println("如需釋放記憶體，可能需要重新建立 map")
}

## 5. 結構 (Struct) - 自訂資料型別

Struct 是 Go 中定義自訂型別的主要方式，可以將相關的資料聚集在一起。

### Struct 的基本定義與初始化

Struct 提供了將相關資料組織在一起的方式，是物件導向程式設計的基礎。

**基本定義**: 使用 `type` 關鍵字定義結構型態。
```go
type Person struct {
    Name    string
    Age     int
    Email   string
}
```

**零值初始化**: 所有欄位都會被初始化為其型態的零值。
```go
var p Person  // Name="", Age=0, Email=""
```

**字面值初始化**: 可以按順序或具名方式初始化。
```go
// 按順序（不推薦）
p1 := Person{"Alice", 25, "alice@example.com"}

// 具名初始化（推薦）
p2 := Person{
    Name:  "Bob",
    Age:   30,
    Email: "bob@example.com",
}
```

**指標初始化**: 建立結構的指標。
```go
p := &Person{
    Name: "Charlie",
    Age:  28,
}
```

**嵌入結構**: Go 支援匿名欄位，實現類似繼承的效果。
```go
type Employee struct {
    Person    // 嵌入 Person
    Salary    float64
    Department string
}
```

**標籤 (Tags)**: 為欄位添加元資料，常用於序列化。
```go
type User struct {
    ID   int    `json:"id" db:"user_id"`
    Name string `json:"name" db:"username"`
}
```

In [ ]:
//go:build ignore
package main

import (
    "fmt"
    "time"
)

// 1. 基本 struct 定義
type Person struct {
    Name    string
    Age     int
    Email   string
    IsAdult bool
}

// 2. 帶標籤的 struct（用於序列化等）
type User struct {
    ID       int    `json:"id" db:"user_id"`
    Username string `json:"username" db:"username"`
    Password string `json:"-" db:"password"` // json:"-" 表示不序列化
    Created  time.Time `json:"created_at" db:"created_at"`
}

// 3. 嵌入結構
type Address struct {
    Street  string
    City    string
    ZipCode string
}

type Employee struct {
    Person  // 嵌入 Person
    Address // 嵌入 Address
    Salary  float64
    Department string
}

func main() {
    // 各種初始化方式
    fmt.Println("=== Struct 初始化方式 ===")
    
    // 1. 零值初始化
    var p1 Person
    fmt.Printf("零值: %+v\n", p1)
    
    // 2. 按欄位順序初始化
    p2 := Person{"Alice", 25, "alice@example.com", true}
    fmt.Printf("按順序: %+v\n", p2)
    
    // 3. 具名欄位初始化（推薦）
    p3 := Person{
        Name:    "Bob",
        Age:     30,
        Email:   "bob@example.com",
        IsAdult: true,
    }
    fmt.Printf("具名欄位: %+v\n", p3)
    
    // 4. 部分初始化
    p4 := Person{
        Name: "Charlie",
        Age:  20,
        // Email 和 IsAdult 使用零值
    }
    fmt.Printf("部分初始化: %+v\n", p4)
    
    // 5. 指標初始化
    p5 := &Person{
        Name:    "David",
        Age:     35,
        Email:   "david@example.com",
        IsAdult: true,
    }
    fmt.Printf("指標: %+v\n", p5)
    
    // 6. 嵌入結構的初始化
    fmt.Println("\n=== 嵌入結構 ===")
    
    emp := Employee{
        Person: Person{
            Name:    "Alice",
            Age:     28,
            Email:   "alice@company.com",
            IsAdult: true,
        },
        Address: Address{
            Street:  "123 Main St",
            City:    "New York",
            ZipCode: "10001",
        },
        Salary:     75000,
        Department: "Engineering",
    }
    
    fmt.Printf("員工: %+v\n", emp)
    
    // 訪問嵌入欄位（可以直接訪問）
    fmt.Printf("姓名: %s\n", emp.Name)    // 等同於 emp.Person.Name
    fmt.Printf("城市: %s\n", emp.City)    // 等同於 emp.Address.City
    fmt.Printf("薪水: %.2f\n", emp.Salary)
}

### Struct 方法與接收者

Go 的方法系統讓結構能夠擁有行為，是實現物件導向程式設計的關鍵機制。

**方法定義**: 方法是附加到特定型態的函數，透過接收者（receiver）與型態關聯。
```go
func (r Rectangle) Area() float64 {
    return r.Width * r.Height
}
```

**值接收者**: 方法接收結構的副本，無法修改原結構。
```go
func (p Person) GetName() string {
    return p.Name  // 唯讀操作
}
```

**指標接收者**: 方法接收結構的指標，可以修改原結構。
```go
func (p *Person) SetAge(age int) {
    p.Age = age  // 可以修改
}
```

**接收者選擇規則**:
- 需要修改結構：使用指標接收者
- 結構很大：使用指標接收者避免複製
- 一致性：同一型態的方法應使用相同類型的接收者

**方法集與介面**: 值型態和指標型態有不同的方法集，影響介面實作。
```go
type Shape interface {
    Area() float64
}

// Rectangle 實作 Shape 介面
var s Shape = Rectangle{Width: 10, Height: 5}
```

**方法調用語法**: Go 會自動處理值和指標之間的轉換。
```go
rect := Rectangle{Width: 10, Height: 5}
rectPtr := &rect

// 以下調用都是有效的
rect.Area()     // 值調用
rectPtr.Area()  // 指標調用（自動解引用）
rect.SetDimensions(8, 6)    // 值調用（自動取址）
rectPtr.SetDimensions(8, 6) // 指標調用
```

In [ ]:
//go:build ignore
package main

import (
    "fmt"
    "math"
)

// 定義結構
type Rectangle struct {
    Width  float64
    Height float64
}

type Circle struct {
    Radius float64
}

// 值接收者方法 - 不能修改原結構
func (r Rectangle) Area() float64 {
    return r.Width * r.Height
}

func (r Rectangle) Perimeter() float64 {
    return 2 * (r.Width + r.Height)
}

// 值接收者方法（嘗試修改，但不會影響原結構）
func (r Rectangle) ScaleValue(factor float64) {
    r.Width *= factor
    r.Height *= factor
    // 只修改副本，不影響原結構
}

// 指標接收者方法 - 可以修改原結構
func (r *Rectangle) ScalePointer(factor float64) {
    r.Width *= factor
    r.Height *= factor
}

func (r *Rectangle) SetDimensions(width, height float64) {
    r.Width = width
    r.Height = height
}

// Circle 的方法
func (c Circle) Area() float64 {
    return math.Pi * c.Radius * c.Radius
}

func (c Circle) Circumference() float64 {
    return 2 * math.Pi * c.Radius
}

func (c *Circle) Scale(factor float64) {
    c.Radius *= factor
}

// 定義介面
type Shape interface {
    Area() float64
}

// 使用介面的函數
func PrintArea(s Shape) {
    fmt.Printf("面積: %.2f\n", s.Area())
}

func main() {
    // 建立結構實例
    fmt.Println("=== 值接收者 vs 指標接收者 ===")
    
    rect := Rectangle{Width: 10, Height: 5}
    fmt.Printf("原始矩形: %+v\n", rect)
    fmt.Printf("面積: %.2f\n", rect.Area())
    fmt.Printf("周長: %.2f\n", rect.Perimeter())
    
    // 值接收者方法
    rect.ScaleValue(2)
    fmt.Printf("ScaleValue後: %+v\n", rect) // 沒有改變
    
    // 指標接收者方法
    rect.ScalePointer(2)
    fmt.Printf("ScalePointer後: %+v\n", rect) // 已改變
    
    // 2. 方法集與介面實作
    fmt.Println("\n=== 介面實作 ===")
    
    circle := Circle{Radius: 3}
    
    // 兩個結構都實作了 Shape 介面
    PrintArea(rect)
    PrintArea(circle)
    
    // 3. 指標與值的方法調用
    fmt.Println("\n=== 方法調用語法 ===")
    
    rect2 := Rectangle{Width: 6, Height: 4}
    rectPtr := &rect2
    
    // 值調用方法（Go 會自動處理）
    fmt.Printf("值調用面積: %.2f\n", rect2.Area())
    fmt.Printf("指標調用面積: %.2f\n", rectPtr.Area()) // 自動解引用
    
    // 指標方法調用
    rect2.ScalePointer(1.5)    // 自動取址
    rectPtr.ScalePointer(1.5)  // 直接調用
    
    fmt.Printf("縮放後: %+v\n", rect2)
    
    // 4. 方法鏈
    fmt.Println("\n=== 方法鏈模式 ===")
    
    type Builder struct {
        data []string
    }
    
    func (b *Builder) Add(item string) *Builder {
        b.data = append(b.data, item)
        return b
    }
    
    func (b *Builder) Build() []string {
        return b.data
    }
    
    builder := &Builder{}
    result := builder.Add("first").Add("second").Add("third").Build()
    
    fmt.Printf("建構結果: %v\n", result)
}

### Struct 的進階特性

掌握 struct 的進階特性，能夠設計出更靈活和高效的資料結構。

**匿名結構**: 在需要臨時資料結構時，可以定義匿名結構。
```go
config := struct {
    Host string
    Port int
    SSL  bool
}{
    Host: "localhost",
    Port: 8080,
    SSL:  false,
}
```

**結構比較**: 如果所有欄位都是可比較的，結構就是可比較的。
```go
type Point struct{ X, Y int }
p1 := Point{1, 2}
p2 := Point{1, 2}
fmt.Println(p1 == p2)  // true

// 包含不可比較欄位的結構不能比較
type Container struct {
    Data []int  // slice 不可比較
}
// c1 == c2  // 編譯錯誤
```

**結構標籤與反射**: 標籤提供元資料，常與反射一起使用。
```go
type User struct {
    ID   int    `json:"id" validate:"required"`
    Name string `json:"name" validate:"min=2,max=50"`
}
```

**複雜嵌入**: 多重嵌入和方法重寫（隱藏）。
```go
type Base struct {
    ID int
}
func (b Base) GetID() int { return b.ID }

type Derived struct {
    Base
    Extra string
}
// Derived 自動擁有 GetID 方法
```

**記憶體佈局優化**: 欄位順序影響記憶體使用，大型欄位應放在前面。
```go
// 不好的對齊（可能浪費記憶體）
type BadStruct struct {
    A bool   // 1 byte + 7 padding
    B int64  // 8 bytes  
    C bool   // 1 byte + 7 padding
}

// 好的對齊
type GoodStruct struct {
    B int64  // 8 bytes
    A bool   // 1 byte
    C bool   // 1 byte + 6 padding
}
```

In [ ]:
//go:build ignore
package main

import (
    "encoding/json"
    "fmt"
    "reflect"
)

// 1. 匿名結構
func demonstrateAnonymousStruct() {
    fmt.Println("=== 匿名結構 ===")
    
    // 匿名結構定義和初始化
    config := struct {
        Host string
        Port int
        SSL  bool
    }{
        Host: "localhost",
        Port: 8080,
        SSL:  false,
    }
    
    fmt.Printf("設定: %+v\n", config)
    
    // 匿名結構切片
    users := []struct {
        Name string
        Age  int
    }{
        {"Alice", 25},
        {"Bob", 30},
    }
    
    fmt.Printf("使用者: %+v\n", users)
}

// 2. 結構比較
type Comparable struct {
    ID   int
    Name string
}

type NotComparable struct {
    ID   int
    Data []string // slice 使結構不可比較
}

func demonstrateStructComparison() {
    fmt.Println("\n=== 結構比較 ===")
    
    // 可比較的結構
    c1 := Comparable{ID: 1, Name: "Alice"}
    c2 := Comparable{ID: 1, Name: "Alice"}
    c3 := Comparable{ID: 2, Name: "Bob"}
    
    fmt.Printf("c1 == c2: %v\n", c1 == c2) // true
    fmt.Printf("c1 == c3: %v\n", c1 == c3) // false
    
    // 不可比較的結構
    n1 := NotComparable{ID: 1, Data: []string{"a", "b"}}
    n2 := NotComparable{ID: 1, Data: []string{"a", "b"}}
    
    // fmt.Printf("n1 == n2: %v\n", n1 == n2) // 編譯錯誤！
    
    // 使用反射比較不可比較的結構
    equal := reflect.DeepEqual(n1, n2)
    fmt.Printf("DeepEqual(n1, n2): %v\n", equal)
}

// 3. 結構標籤與反射
type TaggedStruct struct {
    PublicField  string `json:"public_field" validate:"required"`
    privateField string `json:"-" validate:"optional"`
    Email        string `json:"email" validate:"email"`
}

func demonstrateStructTags() {
    fmt.Println("\n=== 結構標籤 ===")
    
    // JSON 序列化
    ts := TaggedStruct{
        PublicField:  "public",
        privateField: "private",
        Email:        "test@example.com",
    }
    
    jsonData, _ := json.Marshal(ts)
    fmt.Printf("JSON: %s\n", jsonData)
    
    // 反射讀取標籤
    t := reflect.TypeOf(ts)
    for i := 0; i < t.NumField(); i++ {
        field := t.Field(i)
        jsonTag := field.Tag.Get("json")
        validateTag := field.Tag.Get("validate")
        
        fmt.Printf("欄位: %s, json:%s, validate:%s\n", 
                   field.Name, jsonTag, validateTag)
    }
}

// 4. 複雜的嵌入結構
type Base struct {
    ID   int
    Name string
}

func (b Base) GetInfo() string {
    return fmt.Sprintf("ID: %d, Name: %s", b.ID, b.Name)
}

type Derived struct {
    Base  // 嵌入
    Extra string
}

// 方法重寫（實際上是隱藏）
func (d Derived) GetInfo() string {
    return fmt.Sprintf("%s, Extra: %s", d.Base.GetInfo(), d.Extra)
}

type MultipleEmbedding struct {
    Base
    Comparable
    Value float64
}

func demonstrateComplexEmbedding() {
    fmt.Println("\n=== 複雜嵌入 ===")
    
    // 單一嵌入
    d := Derived{
        Base:  Base{ID: 1, Name: "test"},
        Extra: "additional",
    }
    
    fmt.Printf("派生方法: %s\n", d.GetInfo())
    fmt.Printf("基底方法: %s\n", d.Base.GetInfo())
    
    // 多重嵌入
    me := MultipleEmbedding{
        Base:       Base{ID: 1, Name: "multi"},
        Comparable: Comparable{ID: 2, Name: "comp"},
        Value:      3.14,
    }
    
    // 存取可能有衝突的欄位需要明確指定
    fmt.Printf("Base.ID: %d, Comparable.ID: %d\n", me.Base.ID, me.Comparable.ID)
    // fmt.Printf("ID: %d\n", me.ID) // 編譯錯誤！模糊的選擇器
}

// 5. 結構的記憶體佈局
func demonstrateMemoryLayout() {
    fmt.Println("\n=== 記憶體佈局 ===")
    
    type BadAlignment struct {
        A bool   // 1 byte
        B int64  // 8 bytes
        C bool   // 1 byte
        D int32  // 4 bytes
    }
    
    type GoodAlignment struct {
        B int64  // 8 bytes
        D int32  // 4 bytes
        A bool   // 1 byte
        C bool   // 1 byte
        // padding: 2 bytes
    }
    
    fmt.Printf("BadAlignment 大小: %d bytes\n", reflect.TypeOf(BadAlignment{}).Size())
    fmt.Printf("GoodAlignment 大小: %d bytes\n", reflect.TypeOf(GoodAlignment{}).Size())
}

func main() {
    demonstrateAnonymousStruct()
    demonstrateStructComparison()
    demonstrateStructTags()
    demonstrateComplexEmbedding()
    demonstrateMemoryLayout()
}

## 總結

### 複合型態選擇指南

| 型態 | 使用時機 | 優點 | 注意事項 |
|------|----------|------|----------|
| **Array** | 固定大小、高效能需求 | 記憶體連續、編譯時確定大小 | 長度固定、值語意複製成本高 |
| **Slice** | 動態大小的集合 | 靈活、高效、內建支援豐富 | 共享底層陣列的陷阱 |
| **String** | 文字處理 | 不可變、UTF-8 原生支援 | 修改需要轉換、記憶體重分配 |
| **Map** | 鍵值對查找 | O(1) 查找、動態增長 | 非併發安全、迭代順序不固定 |
| **Struct** | 自訂資料模型 | 型別安全、方法綁定、組合靈活 | 記憶體對齊、可比較性限制 |

### 最佳實務要點

1. **效能考量**
   - 預分配 slice 容量避免多次重分配
   - 使用 `strings.Builder` 進行字串拼接
   - 考慮 struct 欄位順序以優化記憶體對齊

2. **安全性**
   - 使用完整切片語法避免意外的底層陣列共享
   - 檢查 map 的 nil 狀態
   - 使用 comma-ok 模式檢查 map 鍵存在性

3. **可維護性**
   - 使用具名欄位初始化 struct
   - 為 struct 欄位添加適當的標籤
   - 優先使用組合而非複雜的嵌入層次

4. **併發安全**
   - Map 需要額外的同步機制
   - Slice 的 append 操作在併發環境下需要保護
   - String 天然併發安全（不可變）

透過深入理解這些複合型態，你將能夠設計出高效、安全且易維護的 Go 程式。